### Kombinasi 6 :
* Delete Duplicate
* Impute Null pake median/mode
* Outlier capping pake Log
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('../UFC_train.csv')

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [3]:
# Delete duplicate
print(f"Shape before dropping duplicates : {df.shape}") # Before dropping duplicates
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}") # After dropping duplicates

Shape before dropping duplicates : (5410, 144)
Shape after dropping duplicates: (5410, 144)


In [4]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [5]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [6]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [7]:
df_number_imputed = df_number.fillna(df_number.median())

In [8]:
# Drop R_fighter, B_fighter, Referee, location, and date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [9]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_number_imputed], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0,0,0,0,0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0,4,0,1,0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0,0,0,0,0,185.42,182.88,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1,5,6,3,0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0,2,3,0,1,160.02,167.64,135.0,34.0,31.0


In [10]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [11]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [12]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [13]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [14]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  561
B_avg_opp_KD             1142
B_avg_SIG_STR_pct         448
B_avg_opp_SIG_STR_pct     425
B_avg_TD_pct              186
                         ... 
R_Height_cms               19
R_Reach_cms                76
R_Weight_lbs              301
B_age                      31
R_age                      74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.369685767097966%
B_avg_opp_KD = 21.109057301293902%
B_avg_SIG_STR_pct = 8.280961182994455%
B_avg_opp_SIG_STR_pct = 7.8558225508317925%
B_avg_TD_pct = 3.4380776340110906%
B_avg_opp_TD_pct = 6.210720887245841%
B_avg_SUB_ATT = 8.22550831792976%
B_avg_opp_SUB_ATT = 5.970425138632162%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 6.358595194085027%
B_avg_SIG_STR_landed = 5.582255083179297%
B_avg_opp_SIG_STR_att = 6.22920517560074%
B_avg_opp_SIG_STR_landed = 6.598890942698706%
B_avg_TOTAL_STR_att = 4.787430683918669%
B_a

In [15]:
# Add a constant value to make all values positive
df_number_positive = df_number + abs(df_number.min()) + 1

df_number_log = np.log(df_number_positive)

In [16]:
df_number_log

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,0.145182,0.117783,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572
1,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,0.000000,...,0.000000,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023
2,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,0.145182,0.117783,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.825469,5.817944,5.655992,3.850148,3.931826
3,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,0.000000,...,0.693147,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984
4,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,0.405465,...,0.000000,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,0.117783,0.000000,0.392886,0.435024,0.543051,0.361165,0.485508,0.318454,0.810930,0.117783,...,0.000000,0.693147,0.000000,0.000000,0.000000,5.795024,5.810362,5.602119,3.806662,3.891820
5406,0.015504,0.000000,0.325686,0.272796,0.478694,0.183688,0.441275,0.286705,0.009718,0.119518,...,0.000000,1.609438,0.693147,1.791759,0.000000,5.825469,5.817944,5.655992,3.850148,3.988984
5407,0.223144,0.223144,0.471565,0.415415,0.285179,0.285179,0.223144,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.795024,5.802722,5.564520,3.761200,3.912023
5408,0.628609,0.117783,0.433810,0.355049,0.030772,0.149497,0.000000,0.015504,0.000000,0.000000,...,0.693147,1.609438,0.693147,0.000000,0.000000,5.795024,5.810362,5.602119,3.828641,3.988984


In [17]:
outliers = dict(check_outlier(df_number_log).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number_log[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 503
B_avg_opp_KD             927
B_avg_SIG_STR_pct        442
B_avg_opp_SIG_STR_pct    407
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               76
R_Weight_lbs             210
B_age                     20
R_age                     65
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 9.297597042513864%
B_avg_opp_KD = 17.13493530499076%
B_avg_SIG_STR_pct = 8.170055452865064%
B_avg_opp_SIG_STR_pct = 7.523105360443623%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 4.085027726432532%
B_avg_SUB_ATT = 4.621072088724584%
B_avg_opp_SUB_ATT = 3.6229205175600745%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 7.707948243992606%
B_avg_SIG_STR_landed = 6.820702402957486%
B_avg_opp_SIG_STR_att = 7.8927911275415905%
B_avg_opp_SIG_STR_landed = 7.911275415896488%
B_avg_TOTAL_STR_att = 8.059149722735674%
B_avg_TOTAL_STR_landed = 7.6

In [18]:
df = pd.concat([df_object, df_number_log], axis=1)

In [19]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [20]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,...,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572,male
1,Lightweight,Orthodox,Orthodox,Blue,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,...,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023,male
2,Welterweight,Orthodox,Orthodox,Blue,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,...,0.000000,0.000000,0.000000,0.000000,5.825469,5.817944,5.655992,3.850148,3.931826,male
3,Welterweight,Southpaw,Orthodox,Blue,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,...,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984,male
4,Bantamweight,Orthodox,Orthodox,Red,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,...,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826,male


In [21]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [22]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [23]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [24]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [25]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [26]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,...,0.000000,0.000000,0.000000,0.000000,5.817944,5.802722,5.707110,3.806662,3.713572,1
1,5,Orthodox,Orthodox,Blue,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,...,1.609438,0.000000,0.693147,0.000000,5.810362,5.832938,5.655992,3.828641,3.912023,1
2,6,Orthodox,Orthodox,Blue,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,...,0.000000,0.000000,0.000000,0.000000,5.825469,5.817944,5.655992,3.850148,3.931826,1
3,6,Southpaw,Orthodox,Blue,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,...,1.791759,1.945910,1.386294,0.000000,5.802722,5.825469,5.655992,3.850148,3.988984,1
4,3,Orthodox,Orthodox,Red,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,...,1.098612,1.386294,0.000000,0.693147,5.747544,5.771566,5.525453,3.970292,3.931826,1


In [27]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()

,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,0.145182,0.117783,0.000000,...,False,False,False,False,False,True,False,False,False,True
1,5,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,...,False,False,False,True,False,False,False,True,False,False
2,6,0.015504,0.000000,0.371564,0.355925,0.223144,0.182322,0.145182,0.117783,0.000000,...,False,False,False,True,False,False,False,True,False,False
3,6,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,...,True,False,False,True,False,False,False,True,False,False
4,3,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,...,False,False,False,True,False,False,False,False,False,True


In [28]:
df.to_csv('../Without Feature Selection/UFC_kombinasi6.csv', index=False)

In [29]:
from sklearn.feature_selection import SelectKBest, f_regression
# Perform feature selection using SelectKBest
selector = SelectKBest(k=5, score_func=f_regression)
X = df.drop('B_Reach_cms', axis=1) 
y = df['B_Reach_cms'] 
X_selected = selector.fit_transform(X, y)

# Get the selected feature names
selected_feature_names = X.columns[selector.get_support()]
print(selected_feature_names)
# Create a new dataframe with only the selected features
df_selected = pd.DataFrame(X_selected, columns=selected_feature_names)
df_selected.head()

Index(['weight_class', 'B_Height_cms', 'B_Weight_lbs', 'R_Height_cms',
       'R_Weight_lbs'],
      dtype='object')


,weight_class,B_Height_cms,B_Weight_lbs,R_Height_cms,R_Weight_lbs
0,7.0,5.817944,5.707110,5.817944,5.707110
1,5.0,5.802722,5.602119,5.810362,5.655992
2,6.0,5.817944,5.655992,5.825469,5.655992
3,6.0,5.795024,5.707110,5.802722,5.655992
4,3.0,5.763622,5.525453,5.747544,5.525453


In [30]:
print(df.shape)

(5410, 149)


In [31]:
df_selected['B_Reach_cms'] = y

In [32]:
df_selected.to_csv('../Punya Andi/UFC_kombinasi6.csv', index=False)